In [3]:
import pandas as pd
import xmlrpc.client

In [ ]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")


tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

test['nom_produccion'] = test['producto'].str.extract(r"^(\S+)")

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)

pareto_prod['nom_produccion'] = pareto_prod['producto'].str.extract(r"^(\S+)")


pareto_merge = pareto_prod[["nom_produccion","Clasificacion"]]

In [10]:
test = test.rename(columns={"total_nbr_minimo":"tiempo_estimado_estandard"})